In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, PersistenceImage 
from gtda.plotting import plot_point_cloud
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns
from hausdorff import hausdorff_distance
import numpy as np
import pickle as pickle
import gudhi as gd  
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import norm
from sklearn.decomposition import KernelPCA
import plotly.io as pio
import random
import h5py
import cv2
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Importing this for 3D plotting
import umap.umap_ as umap  # Correcting the import for UMAP


%matplotlib inline

In [ ]:
# choose individuum
subject = "m292"

# Load data

In [ ]:
label_list = [0, 1, 2, 3, 4]

In [ ]:
# All features
feature_df  = pd.read_csv("Features/All_Features.csv")

In [ ]:
# Load persistence diagrams
persistence_diagrams = np.load("Embeddings_and_Persistence_Diagrams/"+str(subject)+"/Persistence_Diagrams.npy", allow_pickle=True).item()# TODO do this in Preprocessing_And_Computing_...

In [ ]:
extended_persistence_diagrams  = np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Extended_Persistence_Diagrams.npz', \
    allow_pickle=True)

In [ ]:
dataframes = {}

label_list = [0,1,2,3,4] 

n_folds = 5

for label in label_list:
    filename = "Data/"+str(subject)+"/run0"+str(label)+"/Brain_Imaging_Data.h5"
    file = h5py.File(filename,'r')
    dataframes[label] = file['Data']

In [ ]:
most_representative_diagrams = {}

for label in label_list:
    most_representative_diagrams["Label_"+str(label)] =  np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Most_Representative_Diagrams_(Extended_Diagrams)_Label_'+str(label)+'.npy', \
                allow_pickle=True).item()["Label_"+str(label)] # .item() to convert the dtype to dict again


In [ ]:
# TODO delete
most_representative_diagrams =  np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Most_Representative_Diagrams_(Extended_Diagrams)_Label_0+1.npy', \
                allow_pickle=True).item() # .item() to convert the dtype to dict again


most_representative_diagrams["Label_4"] =  np.load('Embeddings_and_Persistence_Diagrams/'+str(subject)+'/Most_Representative_Diagrams_(Extended_Diagrams)_Label_4.npy', \
                allow_pickle=True).item()["Label_4"]  # .item() to convert the dtype to dict again


# Analyze Original Data

In [ ]:
def plot_correlation_with_std(labels, mean_values, std_values, y_axis_name):
    # Create scatter plot with error bars
    plt.errorbar(labels, mean_values, yerr=std_values, fmt='o', color='darkblue', ecolor='skyblue', capsize=4, markersize=8)

    # Set labels and title
    plt.xlabel('Labels', fontsize=14)
    plt.ylabel(y_axis_name, fontsize=14)

    # Fit linear regression model
    X = np.array(labels).reshape(-1, 1)  # Reshape to 2D array for sklearn
    model = LinearRegression()
    model.fit(X, mean_values)

    # Predict y-values using the linear regression model
    regression_line = model.predict(X)

    # Plot linear regression line
    plt.plot(labels, regression_line, color='indianred', linestyle='-', label='Ideal linear relationship', linewidth = 0.6)

    # Calculate extended line coordinates
    x_extended = np.array([-1, 5]).reshape(-1, 1)
    y_extended = model.predict(x_extended)

    # Plot extended line
    plt.plot(x_extended, y_extended, color='indianred', linestyle='--', linewidth = 0.6)

    # Set font size for ticks
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    plt.xlim(-0.5, 4.5)


    # Save the plot as a PNG file
    plt.tight_layout()  
    plt.savefig('Plots/'+y_axis_name+'.png')

    # Show plot
    plt.show()


In [ ]:
brightnesses = {}

for label in label_list:
    brightnesses[label] = []
    for image in dataframes[label]:
        # Convert image to grayscale

        # Calculate average pixel intensity
        average_intensity = cv2.mean(image)[0]

        brightnesses[label].append(average_intensity)

    

In [ ]:
mean_intentities = []
intensities_stds = []

for label in label_list:
    mean_intentities.append(np.mean(brightnesses[label]))
    intensities_stds.append(np.std(brightnesses[label]))


In [ ]:
labels = ["0", "1", "2", "3", "4"]


plot_correlation_with_std(labels, mean_intentities, intensities_stds, "Mean Intensity")


# Dissimilarity Matrix

In [ ]:
# TODO create helper functions for these

def find_largest_dimension(persistence_diagrams, label_list):
    """
    Find the length of the longest list in a list of lists.

    Parameters:
    - persistence_diagrams (dictionary): List of persistence diagrams, with labels as keys.
    - label_list (list): List of all labels.

    Returns:
    - longest_lengths_each_dimension (dictionary): Highest amount of holes in a persistence diagram for each dimension.
    """

    # Initialize highest number of holes of each dimension for each label
    longest_lengths_each_dimension = {}
    for label in label_list:
        longest_lengths_each_dimension["Label_"+str(label)] = {}

        for hole_dim in range(3):
            longest_lengths_each_dimension["Label_"+str(label)]["Hole_Dim_"+str(hole_dim)] = 0

    for label in label_list:

        # Only look at persistence diagrams of current label
        persistence_diagrams_with_label = persistence_diagrams["Label_"+str(label)]


        # For each hole dimension (between 0 and 2), find the largest amount of holes
        # of this dimension which there is in one of the persistence diagrams
        
        for hole_dim in range(3):

            # for the current hole dimension, count the amount of holes in each diagram
            for diagram in persistence_diagrams_with_label:
                current_number_of_holes_of_hole_dim = 0 # Initialize
                
                for hole in diagram[0]: # the holes are nested, so take diagram[0]
                    # only take those holes of the current hole dimension into account
                    # (the hole dimension is indicated by the 3rd entry of the hole)
                    if hole[2] == hole_dim:
                        current_number_of_holes_of_hole_dim += 1

                # If the amount of holes of the current diagram is higher than the 
                # previous highest number of holes, redefine the highest number
                if current_number_of_holes_of_hole_dim > longest_lengths_each_dimension["Label_"+str(label)]["Hole_Dim_"+str(hole_dim)]:
                    longest_lengths_each_dimension["Label_"+str(label)]["Hole_Dim_"+str(hole_dim)] = current_number_of_holes_of_hole_dim

    # Because this is what we want in this case, compute the overall longest dimensions which we will use for all labels
    longest_lengths_each_dimension_overall = {key: max(label_dict[key] for label_dict in longest_lengths_each_dimension.values())
    for key in ['Hole_Dim_0', 'Hole_Dim_1', 'Hole_Dim_2']}
    
    return longest_lengths_each_dimension_overall


def extend_diagrams_to_largest_dimensions(persistence_diagrams, desired_lengths_each_dimension, label_list):
    """
    Extend all persistence diagrams to the largest dimension 
    by appending (0, 0, hole_dim) tuples.

    Parameters:
    - persistence_diagrams (dict): List of persistence_diagrams of each label.
    - longest_lengths_each_dimension_overall (dict): Contains the wished amount of holes for each dimension.
    - label_list (list): List of all labels.

    Returns:
    - extended_persistence_diagrams (dict): List of extended eprsistence_diagrams for each label.
    """

    # Initialize dictionary of extended persistence diagrams (with labels as keys)
    extended_persistence_diagrams = {}

    for label in label_list:

        # Initialize extended persistence diagrams for current label
        extended_persistence_diagrams["Label_"+str(label)] = []
        
        # Only look at persistence diagrams of current label
        persistence_diagrams_with_label = persistence_diagrams["Label_"+str(label)]

        # Extend each diagram to the highest number of holes for each dimension
        for diagram in persistence_diagrams_with_label:

            extended_diagram = list(diagram[0])
            
            for hole_dim in range(3):

                # the persistence diagrams are nested, so take diagram[0]                    
                current_number_of_holes_of_hole_dim = sum(1 for hole in diagram[0] if hole[2] == hole_dim)


                holes_to_append = [[0, 0, hole_dim]] * (desired_lengths_each_dimension["Hole_Dim_"+str(hole_dim)] - current_number_of_holes_of_hole_dim)
                extended_diagram = extended_diagram + holes_to_append
                
            extended_persistence_diagrams["Label_"+str(label)].append(extended_diagram)
            
    return extended_persistence_diagrams


In [ ]:
reshaped_barycenters = {}

for label in label_list:
    reshaped_barycenters["Label_"+str(label)] = [most_representative_diagrams["Label_"+str(label)]]

In [ ]:
longest_lengths_each_dimension_overall = find_largest_dimension(reshaped_barycenters, label_list)
extended_barycenters = extend_diagrams_to_largest_dimensions(reshaped_barycenters, longest_lengths_each_dimension_overall, label_list)

In [ ]:
reshaped_barycenters = {}

for label in label_list:
    reshaped_barycenters["Label_"+str(label)] = extended_barycenters["Label_"+str(label)][0]

In [ ]:
# Wasserstein barycenter
pairwise_distances = PairwiseDistance(metric="wasserstein").fit_transform(list(reshaped_barycenters.values()))

# plot it with seaborn
plt.figure(figsize=(10,10))
ax = sns.heatmap(pairwise_distances,
    cmap='inferno', 
    linewidth=1,
    xticklabels=[0, 1, 2, 3, 4],
    yticklabels=[0, 1, 2, 3, 4],
    cbar_kws={'shrink': 1}
)


# Change the font size of the x and y tick labels
ax.set_xticklabels(ax.get_xticklabels(), fontsize=18)
ax.set_yticklabels(ax.get_yticklabels(), fontsize=18)

ax.set_xlabel('Label', fontsize=20)
ax.set_ylabel('Label', fontsize=20)


plt.savefig('Plots/Dissimilarity_Matrix_Barycenters_'+str(subject)+'.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

# Signatures

## Heat Kernel

In [ ]:
HK = HeatKernel(sigma=1000, n_bins=100)

In [ ]:
def heat_kernel_intensity(heatkernel, homology_dimension):
    """ Computes mean intensity of a heatkernel. Only takes positive values because otherwise the mean would
      always be zero.

    Parameters:
    - heatkernel (list of lists): heatkernel of all homology dimensions
    - homology_dimension (int): Which homology dimension to look at (0, 1 or 2)

    Returns:
    - mean intensity of heatkernel of homology dimension homology_dimension
    """
    
    positives =  [x for inner_list in heatkernel[0][homology_dimension] for x in inner_list if x > 0]
    
    return np.mean(positives)

In [ ]:
label = 4
heatkernel = HK.fit_transform(most_representative_diagrams["Label_"+str(label)])

In [ ]:
print("Heat kernel intensity of label "+str(label)+"  for homology dimension 0: " + str(heat_kernel_intensity(heatkernel, 0)))

plotly_params = {
    "layout": {
        "title": {
            "text": "",
            "x": 0.5,  # Center the title horizontally
            "xanchor": "center",  # Anchor the title text at the center
        },
        "width": 800,
        "height": 650,
        "xaxis": {
            "title": "Birth",  # Add label for x-axis
            "titlefont": {"size": 22},  # Set font size for x-axis label
            "tickfont": {"size": 18}  # Set font size for x-axis ticks
        },
        "yaxis": {
            "title": "Death",  # Add label for y-axis
            "titlefont": {"size": 22},  # Set font size for y-axis label
            "tickfont": {"size": 18}  # Set font size for y-axis ticks
        }
    }
}

fig = HK.plot(heatkernel, homology_dimension_idx=0, colorscale='jet', plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Heatkernel of Dimension 0 (Label "+str(label)+").png")

In [ ]:
print("Heat kernel intensity of label "+str(label)+" for homology dimension 1: " + str(heat_kernel_intensity(heatkernel, 1)))

# Parameters for plotting
plotly_params = {
    "layout": {
        "title": {
            "text": "",
            "x": 0.5,  # Center the title horizontally
            "xanchor": "center",  # Anchor the title text at the center
        },
        "width": 800,
        "height": 650,
        "xaxis": {
            "title": "Birth",  # Add label for x-axis
            "titlefont": {"size": 22},  # Set font size for x-axis label
            "tickfont": {"size": 18}  # Set font size for x-axis ticks
        },
        "yaxis": {
            "title": "Death",  # Add label for y-axis
            "titlefont": {"size": 22},  # Set font size for y-axis label
            "tickfont": {"size": 18}  # Set font size for y-axis ticks
        }
    }
}

fig = HK.plot(heatkernel, homology_dimension_idx=1, colorscale='jet', plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Heatkernel of Dimension 1 (Label "+str(label)+").png")

In [ ]:
print("Heat kernel intensity of label "+str(label)+" for homology dimension 2: " + str(heat_kernel_intensity(heatkernel, 2)))

# Parameters for plotting
plotly_params = {
    "layout": {
        "title": {
            "text": "",
            "x": 0.5,  # Center the title horizontally
            "xanchor": "center",  # Anchor the title text at the center
        },
        "width": 800,
        "height": 650,
        "xaxis": {
            "title": "Birth",  # Add label for x-axis
            "titlefont": {"size": 22},  # Set font size for x-axis label
            "tickfont": {"size": 18}  # Set font size for x-axis ticks
        },
        "yaxis": {
            "title": "Death",  # Add label for y-axis
            "titlefont": {"size": 22},  # Set font size for y-axis label
            "tickfont": {"size": 18}  # Set font size for y-axis ticks
        }
    }
}

fig = HK.plot(heatkernel, homology_dimension_idx=2, colorscale='jet', plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Heatkernel of Dimension 2 (Label "+str(label)+").png")

# Choose subset of persistence diagrams to take into account

# Persistence Landscape

In [ ]:
PL = PersistenceLandscape()

In [ ]:
label = 0

landscape = PL.fit_transform(most_representative_diagrams["Label_"+str(label)])

# Parameters for Plotting
plotly_params = {
    "layout": {
        "showlegend": False,  # Hide the legend
        "title": "", # Remove the title
        "height": 500,  # Set the desired height in pixels
        "margin": dict(l=150, r=50, t=50, b=100)  # Adjust margins if needed
    }
}


In [ ]:
# Generate the initial plot
fig = PL.plot(landscape, homology_dimensions=[0, 1, 2], plotly_params=plotly_params)

# Update trace names to change legend text
for i, trace in enumerate(fig.data):
    if i == 0:
        trace.name = "Dim 0"
    elif i == 1:
        trace.name = "Dim 1"
    elif i == 2:
        trace.name = "Dim 2"

# Update layout to add combined axis labels and increase legend font size
fig.update_layout(
    annotations=[
        dict(
            text="Filtration Parameter",
            x=0.5,
            y=-0.2,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=18)
        ),
        dict(
            text="Persistence",
            x=-0.1,
            y=0.5,
            xref="paper",
            yref="paper",
            showarrow=False,
            textangle=-90,
            font=dict(size=18)
        )
    ],
    margin=dict(l=100, r=20, t=100, b=100),  # Adjust margins if necessary
    legend=dict(
        font=dict(
            size=18  # Set the desired font size for the legend
        )
    )
)

fig.show()


# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Brain_Imaging_Landscape (Label "+str(label)+").png")

# Silhouette

In [ ]:
SH = Silhouette()

In [ ]:
label = 4
silhouette = SH.fit_transform(most_representative_diagrams["Label_"+str(label)])


In [ ]:
# Parameters for Plotting
plotly_params = {
    "layout": {
        "showlegend": True,  # Hide the legend
        "title": "", # Remove the title
        "height": 300,  # Set the desired height in pixels
        "margin": dict(l=100, r=50, t=50, b=100),
        "xaxis": {
            "titlefont": {"size": 22},  # Set font size for x-axis label
            "tickfont": {"size": 18}  # Set font size for x-axis ticks
        },
        "yaxis": {
            "title": "Silhouette Value",  # Add label for y-axis
            "titlefont": {"size": 22},  # Set font size for y-axis label
            "tickfont": {"size": 18}  # Set font size for y-axis ticks
        }
    }
}

fig = SH.plot(silhouette, homology_dimensions=[0, 1, 2], plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Brain_Imaging_Silhouette (Label "+str(label)+").png")

# Betti Curve

In [ ]:
BC = BettiCurve()

In [ ]:
# Parameters for Plotting
plotly_params = {
    "layout": {
        "showlegend": True,  # Hide the legend
        "title": "", # Remove the title
        "height": 300,  # Set the desired height in pixels
        "margin": dict(l=100, r=50, t=50, b=100),
        "xaxis": {
            "titlefont": {"size": 22},  # Set font size for x-axis label
            "tickfont": {"size": 18}  # Set font size for x-axis ticks
        },
        "yaxis": {
            "title": "Betti Number",  # Add label for y-axis
            "titlefont": {"size": 22},  # Set font size for y-axis label
            "tickfont": {"size": 18}  # Set font size for y-axis ticks
        }
    }
}


In [ ]:
label = 0
betti_curve = BC.fit_transform_plot(most_representative_diagrams["Label_"+str(label)]) # memory efficient enough

In [ ]:

fig = BC.plot(betti_curve, homology_dimensions=[0, 1, 2], plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)
pio.write_image(fig, "Plots/Brain_Imaging_Betti_Curve (Label "+str(label)+").png")

# Persistence Images

In [ ]:
PI = PersistenceImage(sigma=1000, n_bins=100)

In [ ]:
label = 0
persistence_image = PI.fit_transform(most_representative_diagrams["Label_"+str(label)])

In [ ]:
# Parameters for plotting

plotly_params = {
    "layout": {
        "title": {
            "text": "",
            "x": 0.5,
            "xanchor": "center",
        },
        "width": 500,
        "height": 400,  # Set an appropriate overall plot height
        "xaxis": {
            "title": "Birth",
            "titlefont": {"size": 22},
            "tickfont": {"size": 18},
            "domain": [0.5,0.8]  # Reduce domain to make the y-axis occupy less space

        },
        "yaxis": {
            "title": "Death",
            "titlefont": {"size": 22},
            "tickfont": {"size": 18},
            "range": [5000, 7000],  # Keep the range fixed
            "domain": [0,1]  # Reduce domain to make the y-axis occupy less space
        },
        "margin": {"l": 50, "r": 50, "t": 50, "b": 50}
    }
}


fig = PI.plot(persistence_image, homology_dimension_idx=2, colorscale='jet', plotly_params=plotly_params)

fig.show()

# Save the plot as an image (PNG format)

#pio.write_image(fig, "Plots/Brain_Imaging_Persistence_Image Hom_Dim 2 (Label "+str(label)+").png")

# Dimensionality reduced visualizations of Signature Vectorizations

First of all, combine all (extended persistence diagrams) to later examine the visualizations for clusters.

In [ ]:

def separate_signature_according_to_dimension(signature):
    """
    Separate the landscape vectors according to each dimension.

    Parameters:
    - signature (list): Signature vector.

    Returns:
    - vectors (dict): Dictionary containing lists of vectors for each dimension.
    """
    vectors = {}
    
    for hom_dim in range(2):
        vectors["Dimension_" + str(hom_dim)] = []
        
        for vector_for_single_diagram in signature:
            vectors["Dimension_" + str(hom_dim)].append(vector_for_single_diagram[hom_dim])
    
    return vectors

def compute_umap_embedding(complete_data, data_list, homology_dimension, random_state=1):
    """
    Perform UMAP dimensionality reduction to 3 dimensions.

    Parameters:
    - signature (list): List containing the signature data.

    Returns:
    - embedding (numpy array): The 3-dimensional UMAP embedding.
    """
    # Perform UMAP dimensionality reduction to 3 dimensions


    reducer = umap.UMAP(n_components=3, random_state=random_state)
    
    reducer.fit(complete_data)
    
    embeddings = []

    for label in range(5):
        embeddings.append(reducer.transform(data_list["Label_"+str(label)]["Dimension_"+str(homology_dimension)]))
    
    return embeddings


def visualize_embedding(list_of_embeddings, title="Signature"):
    """
    Visualize two 3-dimensional UMAP embeddings in the same plot.

    Parameters:
    - list_of_embeddings (list): List of arrays with the first 3-dimensional UMAP embedding.
    - title (str): Title for the plot.

    Returns:
    None
    """
    # Visualize the embeddings
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Scatter plots for each label
    for i, embedding in enumerate(list_of_embeddings):
        ax.scatter(embedding[:, 0], embedding[:, 1], embedding[:, 2], label=f'Label {i}', color=['blue', 'blueviolet', 'purple', 'crimson', 'red'][i])

    ax.set_xlabel('Dimension 1', fontsize=18)
    ax.set_ylabel('Dimension 2', fontsize=18)
    ax.set_zlabel('Dimension 3', fontsize=18)
    ax.set_title("")

    ax.set_box_aspect([1,1,1])  # Equal aspect ratio

    plt.tick_params(axis='both', which='major', labelsize=15)
    ax.legend(fontsize=13)

    plt.savefig('Plots/'+str(title)+'.png', bbox_inches='tight')

    plt.show()



def compute_kernel_pca_embedding(complete_data, data_list, homology_dimension, kernel='rbf', n_components=3):
    """
    Compute a Kernel PCA reduced embedding with 3 dimensions.
    
    Parameters:
        data (numpy.ndarray): Input data matrix of shape (n_samples, n_features).
        kernel (str): Kernel function to be used ('rbf', 'poly', 'sigmoid', 'linear').
        n_components (int): Number of components for dimensionality reduction.
        
    Returns:
        numpy.ndarray: Kernel PCA reduced embedding of shape (n_samples, 3).
    """
    # Initialize Kernel PCA with specified kernel function
    kpca = KernelPCA(kernel=kernel, n_components=n_components, random_state=42)

    kpca.fit(complete_data[0][homology_dimension])
    
    # Fit and transform the data
    embeddings = []

    for label in range(5):
        embeddings.append(kpca.transform(data_list["Label_"+str(label)][0][homology_dimension]))
    
    return embeddings
    


## Betti Curve

In [ ]:
BC = BettiCurve()

In [ ]:
curves = {}

for label in range(5):
    curves["Label_"+str(label)] = BC.fit_transform(extended_persistence_diagrams["Label_"+str(label)])

vectors = {}

for label in range(5):
    vectors["Label_"+str(label)]  = separate_signature_according_to_dimension(curves["Label_"+str(label)])

In [ ]:
hom_dim = 0

all_vectors = []

for label in range(5):
    all_vectors.extend(vectors["Label_"+str(label)]["Dimension_"+str(hom_dim)])

# Compute combined representation
embedding_list = compute_umap_embedding(all_vectors, vectors, hom_dim)

fig = visualize_embedding(embedding_list, "BI Dimensionality Reduced Betti Curve of Dimension "+str(hom_dim))


In [ ]:
hom_dim = 1

all_vectors = []

for label in range(5):
    all_vectors.extend(vectors["Label_"+str(label)]["Dimension_"+str(hom_dim)])

# Compute combined representation
embedding_list = compute_umap_embedding(all_vectors, vectors, hom_dim)

fig = visualize_embedding(embedding_list, "BI Dimensionality Reduced Betti Curve of Dimension "+str(hom_dim))
 

## Silhouette

In [ ]:
SH = Silhouette()

In [ ]:
curves = {}

for label in range(5):
    curves["Label_"+str(label)] = SH.fit_transform(random_subsets["Label_"+str(label)])

vectors = {}

for label in range(5):
    vectors["Label_"+str(label)]  = separate_signature_according_to_dimension(curves["Label_"+str(label)])

In [ ]:
hom_dim = 0

all_vectors = []

for label in range(5):
    all_vectors.extend(vectors["Label_"+str(label)]["Dimension_"+str(hom_dim)])

# Compute combined representation
embedding_list = compute_umap_embedding(all_vectors, vectors, hom_dim)

fig = visualize_embedding(embedding_list, "BI Dimensionality Reduced Silhouette of Dimension "+str(hom_dim))


In [ ]:
hom_dim = 1

all_vectors = []

for label in range(5):
    all_vectors.extend(vectors["Label_"+str(label)]["Dimension_"+str(hom_dim)])

# Compute combined representation
embedding_list = compute_umap_embedding(all_vectors, vectors, hom_dim)

fig = visualize_embedding(embedding_list, "BI Dimensionality Reduced Silhouette of Dimension "+str(hom_dim))


## Persistence Landscape

In [ ]:
PL = PersistenceLandscape()

In [ ]:
curves = {}

for label in range(5):
    curves["Label_"+str(label)] = PL.fit_transform(random_subsets["Label_"+str(label)])

vectors = {}

for label in range(5):
    vectors["Label_"+str(label)]  = separate_signature_according_to_dimension(curves["Label_"+str(label)])

In [ ]:
hom_dim = 0

all_vectors = []

for label in range(5):
    all_vectors.extend(vectors["Label_"+str(label)]["Dimension_"+str(hom_dim)])

# Compute combined representation
embedding_list = compute_umap_embedding(all_vectors, vectors, hom_dim)

fig = visualize_embedding(embedding_list, "BI Dimensionality Reduced Persistence Landscape of Dimension "+str(hom_dim))


# Finding Clusters in Features Directly

In [ ]:
# All features
feature_df  = pd.read_csv("Features/All_Features.csv")

In [ ]:
# Choose m292 as subject
subject_df = feature_df[feature_df["Subject"] == 1]

### Set plot parameters ####

num_points_per_label = 74

# Define the colors for each label
colors=['blue', 'blueviolet', 'purple', 'crimson', 'red']


In [ ]:
# Plotting the points
plt.figure(figsize=(6, 6))

# Plot each segment with the corresponding color
for i, color in enumerate(colors):
    start_idx = i * num_points_per_label
    end_idx = start_idx + num_points_per_label
    plt.scatter(subject_df['Persistence Entropy_Dim_0'][start_idx:end_idx],
                subject_df['Persistence Entropy_Dim_1'][start_idx:end_idx],
                color=color, label=f'Label {i}')

# Adding labels and title
plt.xlabel('PE Dim 0', fontsize=18)
plt.ylabel('PE Dim 1', fontsize=18)
plt.title('')

plt.tick_params(axis='both', which='major', labelsize=15)

# Add legend to differentiate the segments
plt.legend()

# Save the plot
plt.savefig('Plots/Persistent Entropy 1 & 2.png', bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
# Plotting the points
plt.figure(figsize=(6, 6))

# Plot each segment with the corresponding color
for i, color in enumerate(colors):
    start_idx = i * num_points_per_label
    end_idx = start_idx + num_points_per_label
    plt.scatter(subject_df['Amplitude_Dim_0'][start_idx:end_idx],
                subject_df['Amplitude_Dim_1'][start_idx:end_idx],
                color=color, label=f'Label {i}')

# Adding labels and title
plt.xlabel('Amplitude Dim 0', fontsize=18)
plt.ylabel('Amplitude Dim 1', fontsize=18)
plt.title('')

plt.tick_params(axis='both', which='major', labelsize=15)

# Add legend to differentiate the labels
plt.legend()

# Save the plot
plt.savefig('Plots/Amplitude 1 & 2.png', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Plotting the points
plt.figure(figsize=(6, 6))

# Plot each segment with the corresponding color
for i, color in enumerate(colors):
    start_idx = i * num_points_per_label
    end_idx = start_idx + num_points_per_label
    plt.scatter(subject_df['AC_coordinate_cc1'][start_idx:end_idx],
                subject_df['AC_coordinate_cc2'][start_idx:end_idx],
                color=color, label=f'Label {i}')

# Adding labels and title
plt.xlabel('AC 1', fontsize=18)
plt.ylabel('AC 2', fontsize=18)
plt.title('')

plt.tick_params(axis='both', which='major', labelsize=15)

# Add legend to differentiate the labels
plt.legend()

# Save the plot
plt.savefig('Plots/Clustering AC 1 & 2.png', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Plotting the points
plt.figure(figsize=(6, 6))

# Plot each segment with the corresponding color
for i, color in enumerate(colors):
    start_idx = i * num_points_per_label
    end_idx = start_idx + num_points_per_label
    plt.scatter(subject_df['AC_coordinate_cc3'][start_idx:end_idx],
                subject_df['AC_coordinate_cc4'][start_idx:end_idx],
                color=color, label=f'Label {i}')

# Adding labels and title
plt.xlabel('AC 3', fontsize=18)
plt.ylabel('AC 4', fontsize=18)
plt.title('')

plt.tick_params(axis='both', which='major', labelsize=15)

# Add legend to differentiate the labels
plt.legend()

# Save the plot
plt.savefig('Plots/Clustering AC 3 & 4.png', bbox_inches='tight')

# Display the plot
plt.show()

In [ ]:
# Plotting the points
plt.figure(figsize=(6, 6))

# Plot each segment with the corresponding color
for i, color in enumerate(colors):
    start_idx = i * num_points_per_label
    end_idx = start_idx + num_points_per_label
    plt.scatter(subject_df['No_Points_Dim_0'][start_idx:end_idx],
                subject_df['No_Points_Dim_1'][start_idx:end_idx],
                color=color, label=f'Label {i}')

# Adding labels and title
plt.xlabel('No Points Dim 0', fontsize=18)
plt.ylabel('No Points Dim 1', fontsize=18)
plt.title('')

plt.tick_params(axis='both', which='major', labelsize=15)

# Add legend to differentiate the labels
plt.legend()

# Save the plot
plt.savefig('Plots/No Points 1 & 2.png', bbox_inches='tight')

# Display the plot
plt.show()